# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846776764444                   -0.70    4.5         
  2   -7.852314679519       -2.26       -1.53    1.0   29.7ms
  3   -7.852615280567       -3.52       -2.55    1.5   34.3ms
  4   -7.852645947678       -4.51       -2.87    2.5   39.5ms
  5   -7.852646477112       -6.28       -3.14    1.0   30.7ms
  6   -7.852646678730       -6.70       -4.10    1.2   31.0ms
  7   -7.852646686640       -8.10       -5.13    2.2   38.7ms
  8   -7.852646686725      -10.07       -5.48    1.5   33.7ms
  9   -7.852646686726      -12.22       -5.47    1.2   31.4ms
 10   -7.852646686730      -11.41       -6.74    1.0   30.0ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846827243147                   -0.70           4.2         
  2   -7.852529400443       -2.24       -1.64   0.80    2.2    331ms
  3   -7.852637286534       -3.97       -2.72   0.80    1.0   27.7ms
  4   -7.852646437011       -5.04       -3.30   0.80    2.0   35.2ms
  5   -7.852646682489       -6.61       -4.16   0.80    1.8   32.9ms
  6   -7.852646686572       -8.39       -4.86   0.80    2.0   34.2ms
  7   -7.852646686723       -9.82       -5.85   0.80    1.8   33.9ms
  8   -7.852646686730      -11.20       -6.57   0.80    2.2   35.8ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.405013e+01     3.481822e+00
 * time: 0.06895995140075684
     1     1.102799e+00     1.921991e+00
 * time: 0.32888197898864746
     2    -1.605068e+00     2.113716e+00
 * time: 0.35839009284973145
     3    -3.944285e+00     1.730618e+00
 * time: 0.4003620147705078
     4    -5.282932e+00     1.463238e+00
 * time: 0.4415099620819092
     5    -6.981249e+00     7.145949e-01
 * time: 0.48272109031677246
     6    -7.295606e+00     6.905351e-01
 * time: 0.5103180408477783
     7    -7.729764e+00     2.006619e-01
 * time: 0.5390410423278809
     8    -7.799664e+00     1.257350e-01
 * time: 0.5682320594787598
     9    -7.829489e+00     8.027929e-02
 * time: 0.597661018371582
    10    -7.842262e+00     5.117779e-02
 * time: 0.6265299320220947
    11    -7.847937e+00     3.955934e-02
 * time: 0.655493974685669
    12    -7.851219e+00     1.767533e-02
 * time: 0.6851019859313965
    13    -7.852256e+00     1.243512e-02
 * time: 0.71441388

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846860480053                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645916084                   -1.64         
  2   -7.852646686730       -6.11       -3.71    2.34s
  3   -7.852646686730      -13.30       -7.25    215ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.066341977444012e-7
|ρ_newton - ρ_scfv| = 1.2855444887966448e-7
|ρ_newton - ρ_dm|   = 6.392577152507202e-10
